# 1.92 Atividade Final - Análise de Sentimento com Regressão Logística
Agora que já vimos a fundo como funciona a Regressão Logística e como podemos utilizá-la para predizer e classificar dados com base em um _dataset_ já anotado, podemos realizar a nossa atividade final. Vamos realizar a importação das nossas bibliotecas e funções, extrair recursos desse texto, aplicar a regressão logística, realizar o teste e então executar uma análise de erros!

In [5]:
# vamos importar o NLTK
import nltk
from os import getcwd

### Instale os pacotes do NLTK

```Python
nltk.download('twitter_samples')
```
```python
nltk.download('stopwords')
```

#### Caso não tenha o arquivo utils.py (com as funções que criamos) em seu computador, ele se encontra nessa mesma pasta
* `process_tweet()`: limpa o texto, transforma-o em palavras separadas (tokenização), remove palavras irrelevantes (stop words) e converte palavras em radicais (stem).
* `build_freqs()`: isso conta quantas vezes uma palavra no 'corpus' (todo o conjunto de tweets) foi associada a um rótulo positivo '1' ou negativo '0', em seguida, constrói o dicionário `freqs`, onde cada chave é uma tupla (palavra, rótulo) e o valor é a contagem de sua frequência dentro do corpus de tweets.

In [7]:
# faça o download (está em nossa subpasta de NLP) e adicione o folder tmp2 como filePath. Vamos utilizar como corpora.

filePath = f"{getcwd()}/../tmp2/"
#filePath = "/../tmp2/" ou esse
nltk.data.path.append(filePath)

In [9]:
import numpy as np
import pandas as pd
from nltk.corpus import twitter_samples 
from utils import process_tweet, build_freqs

### Preparando dados
* O twitter_samples contém subconjuntos de 5.000 tweets positivos, 5.000 tweets negativos e o conjunto completo de 10.000 tweets.
* Vamos selecionar cinco mil tweets positivos e cinco mil tweets negativos.

In [10]:
# selecionando o _dataset_ positivo e negativo em variáveis
all_positive_tweets = twitter_samples.strings('positive_tweets.json')
all_negative_tweets = twitter_samples.strings('negative_tweets.json')

**20% estarão no conjunto de teste e 80% no conjunto de treinamento.**

In [11]:
# dividindo os dados em duas partes, uma para treinamento e outra para teste (conjunto de validação)
test_pos = all_positive_tweets[4000:]
train_pos = all_positive_tweets[:4000]
test_neg = all_negative_tweets[4000:]
train_neg = all_negative_tweets[:4000]

train_x = train_pos + train_neg 
test_x = test_pos + test_neg

**Criando a matriz numpy de labels positivas e negativas.**

In [13]:
# Combinando labels positivos e negativos
train_y = np.append(np.ones((len(train_pos), 1)), np.zeros((len(train_neg), 1)), axis=0)
test_y = np.append(np.ones((len(test_pos), 1)), np.zeros((len(test_neg), 1)), axis=0)

In [14]:
# Print do "shape" do set de treino e teste
print("train_y.shape = " + str(train_y.shape))
print("test_y.shape = " + str(test_y.shape))

train_y.shape = (8000, 1)
test_y.shape = (2000, 1)


* Crie o dicionário de frequência usando a função importada `build_freqs ()`.
     * É altamente recomendável que você abra `utils.py` e leia a função` build_freqs () `para entender o que está fazendo.

```Python
    for y,tweet in zip(ys, tweets):
        for word in process_tweet(tweet):
            pair = (word, y)
            if pair in freqs:
                freqs[pair] += 1
            else:
                freqs[pair] = 1
```
* Observe como o loop "for" externo passa por cada tweet, e o loop "for" interno percorre cada palavra em um tweet.
* O dicionário `freqs` é o dicionário de frequência que está sendo construído.
* A chave é a tupla (word, label), como ("happy", 1) ou ("happy", 0). O valor armazenado para cada chave é a contagem de quantas vezes a palavra "happy" foi associada a um rótulo positivo ou quantas vezes "happy" foi associada a um rótulo negativo.

In [15]:
# crie dicionário de frequências
freqs = build_freqs(train_x, train_y)

# output com type e len
print("type(freqs) = " + str(type(freqs)))
print("len(freqs) = " + str(len(freqs.keys())))

type(freqs) = <class 'dict'>
len(freqs) = 11340


### Processando o tweet
A função fornecida `process_tweet ()` tokeniza o tweet em palavras individuais, remove stop words e aplica lematização.

In [16]:
print('This is an example of a positive tweet: \n', train_x[0])
print('\nThis is an example of the processed version of the tweet: \n', process_tweet(train_x[0]))

This is an example of a positive tweet: 
 #FollowFriday @France_Inte @PKuchly57 @Milipol_Paris for being top engaged members in my community this week :)

This is an example of the processed version of the tweet: 
 ['followfriday', 'top', 'engag', 'member', 'commun', 'week', ':)']


# 1. Regressão logística

### Parte 1.1: Sigmóide
* A função sigmóide é definida como:

$$ h(z) = \frac{1}{1+\exp^{-z}} \tag{1}$$

Ele mapeia a entrada 'z' para um valor que varia entre 0 e 1 e, portanto, pode ser tratado como uma probabilidade.

<div style="width:image width px; font-size:100%; text-align:center;"><img src='images/sigmoid_plot.jpg' alt="alternate text" width="width" height="height" style="width:300px;height:200px;" /> Figura 1 </div>

#### Vamos implementar a função sigmóide
Procure saber sobre o numpy.exp
<ul>
    <li><a href="https://docs.scipy.org/doc/numpy/reference/generated/numpy.exp.html" > numpy.exp </a> </li>
</ul>

In [18]:
def sigmoid(z): 
    '''
    Input:
         z: é a entrada (pode ser um escalar ou uma matriz)
     Output:
         h: o sigmóide de z
    '''

    # calculando sigmóide de z
    h = 1 / (1 + np.exp(-z))  
    
    return h

In [19]:
# Testando a função...
if (sigmoid(0) == 0.5):
    print('SUCCESS!')
else:
    print('Oops!')

if (sigmoid(4.92) == 0.9927537604041685):
    print('CORRECT!')
else:
    print('Oops again!')

SUCCESS!
CORRECT!


### Regressão logística: regressão e um sigmóide

A regressão logística faz uma regressão linear regular e aplica um sigmóide à saída da regressão linear.

Regressão:
$$z = \theta_0 x_0 + \theta_1 x_1 + \theta_2 x_2 + ... \theta_N x_N$$
Observe que os valores $\theta$ são "pesos". Se você fez a especialização em Deep Learning, nos referimos aos pesos com o vetor `w`. Estamos usando uma variável diferente $\theta$ para nos referir aos pesos.

Regressão logística:
$$ h(z) = \frac{1}{1+\exp^{-z}}$$
$$z = \theta_0 x_0 + \theta_1 x_1 + \theta_2 x_2 + ... \theta_N x_N$$
'z' = 'logits'

### Parte 1.2 Função de custo e gradiente
A função de custo usada para regressão logística é a média da perda de log em todos os exemplos de treinamento:

$$J(\theta) = -\frac{1}{m} \sum_{i=1}^m y^{(i)}\log (h(z(\theta)^{(i)})) + (1-y^{(i)})\log (1-h(z(\theta)^{(i)}))\tag{5} $$
* $m$ número de exemplos de treinamento
* $y^{(i)}$ é o label real do i-ésimo exemplo de treinamento.
* $h(z(\theta)^{(i)})$ é a previsão do modelo para o i-ésimo exemplo de treinamento.

A função de perda para um único exemplo de treinamento é:
$$ Loss = -1 \times \left( y^{(i)}\log (h(z(\theta)^{(i)})) + (1-y^{(i)})\log (1-h(z(\theta)^{(i)})) \right)$$

* Todos os valores de $h$ estão entre 0 e 1, então os logs serão negativos. Essa é a razão para o fator -1 aplicado à soma dos dois termos de perda.
* Observe que quando o modelo prevê 1 ($h(z(\theta)) = 1$) e o rótulo $y$ também é 1, a perda para esse exemplo de treinamento é 0.
* Da mesma forma, quando o modelo prevê 0 ($h(z(\theta)) = 0$) e o rótulo real também é 0, a perda para esse exemplo de treinamento é 0.
* No entanto, quando a previsão do modelo está perto de 1 ($h(z(\theta)) = 0.9999$) e o rótulo é 0, o segundo termo da perda de log torna-se um grande número negativo, que é então multiplicado pelo fator geral de -1 para convertê-lo em um valor de perda positivo. $-1 \times (1 - 0) \times log(1 - 0,9999)\approx 9.2$ Quanto mais perto a previsão do modelo chega de 1, maior a perda.

In [20]:
# verifique que quando o modelo prevê perto de 1, mas o rótulo real é 0, a perda é um grande valor positivo
-1 * (1 - 0) * np.log(1 - 0.9999) # perda (loss) = aprox. 9.2

9.210340371976294

* Da mesma forma, se o modelo prevê perto de 0 ($h(z) = 0.0001 $), mas o rótulo real é 1, o primeiro termo na função de perda torna-se um grande número: $-1 \times log(0.0001) \approx 9.2$. Quanto mais próxima de zero a previsão estiver, maior será a perda.

In [21]:
# verifique se quando o modelo prevê perto de 0, mas o rótulo real é 1, a perda é um grande valor positivo
-1 * np.log(0.0001) # perda (loss) é próxima de 9.2

9.210340371976182

#### Atualizando os pesos

Para atualizar seu vetor de peso $ \theta$, você aplicará gradiente descendente para melhorar iterativamente as previsões do seu modelo.
O gradiente da função de custo $ J $ em relação a um dos pesos $ \theta_j $ é:

$$\nabla_{\theta_j}J(\theta) = \frac{1}{m} \sum_{i=1}^m(h^{(i)}-y^{(i)})x_j \tag{5}$$
* 'i' é o índice de todos os exemplos de treinamento 'm'.
* 'j' é o índice do peso $ \theta_j$, então $ x_j $ é a característica associada ao peso $ \theta_j $

* Para atualizar o peso $ \theta_j $, nós o ajustamos subtraindo uma fração do gradiente determinado por $ \alpha $:
$$\theta_j = \theta_j - \alpha \times \nabla_{\theta_j}J(\theta) $$
* A taxa de aprendizado $\alpha $ é um valor que escolhemos para controlar o tamanho de uma única atualização.

## Implementando a função de descida gradiente
* O número de iterações `num_iters` é o número de vezes que você usará todo o conjunto de treinamento.
* Para cada iteração, você calculará a função de custo usando todos os exemplos de treinamento (existem exemplos de treinamento `m`) e para todos os recursos.
* Em vez de atualizar um único peso $ \theta_i $ por vez, podemos atualizar todos os pesos no vetor coluna:
$$\mathbf{\theta} = \begin{pmatrix}
\theta_0
\\
\theta_1
\\ 
\theta_2 
\\ 
\vdots
\\ 
\theta_n
\end{pmatrix}$$
* $\mathbf{\theta}$ tem dimensões (n + 1, 1), onde 'n' é o número de recursos, e há mais um elemento para o termo de polarização $ \theta_0 $ (observe que o valor do recurso correspondente $ \mathbf {x_0} $ é 1 )
* Os 'logits', 'z', são calculados multiplicando a matriz de características 'x' com o vetor de peso 'theta'. $ z =\mathbf{x}\mathbf{\theta}$
    * $\mathbf{x}$ has dimensions (m, n+1) 
    * $\mathbf{\theta}$: has dimensions (n+1, 1)
    * $\mathbf{z}$: has dimensions (m, 1)
* A previsão 'h' é calculada pela aplicação do sigmóide a cada elemento em 'z': $ h (z) = sigmóide (z) $, e tem dimensões (m, 1).
* A função de custo $ J $ é calculada tomando o produto escalar dos vetores 'y' e 'log (h)'. Uma vez que 'y' e 'h' são vetores coluna (m, 1), transponha o vetor para a esquerda, de modo que a multiplicação da matriz de um vetor linha com vetor coluna execute o produto escalar.
$$J = \frac{-1}{m} \times \left(\mathbf{y}^T \cdot log(\mathbf{h}) + \mathbf{(1-y)}^T \cdot log(\mathbf{1-h}) \right)$$
* A atualização de theta também é vetorizada. Como as dimensões de $ \mathbf {x} $ são (m, n + 1), e ambos $ \mathbf {h} $ e $ \mathbf {y} $ são (m, 1), precisamos transpor o $ \mathbf{x}$ e coloque-o à esquerda para realizar a multiplicação da matriz, que então produz a resposta (n + 1, 1) que precisamos:
$$\mathbf{\theta} = \mathbf{\theta} - \frac{\alpha}{m} \times \left( \mathbf{x}^T \cdot \left( \mathbf{h-y} \right) \right)$$


- **Vamos usar np.dot para multiplicação de matrizes.** 
- **Para garantir que a fração -1 / m seja um valor decimal, lance o numerador ou denominador (ou ambos), como `float (1)`, ou escreva `1.` para a versão flutuante de 1.**

In [23]:
def gradientDescent(x, y, theta, alpha, num_iters):
    '''
     Input:
         x: matriz de recursos que é (m, n + 1)
         y: rótulos correspondentes da matriz de entrada x, dimensões (m, 1)
         theta: vetor peso da dimensão (n + 1,1)
         alfa: taxa de aprendizagem
         num_iters: número de iterações para as quais você deseja treinar seu modelo
     Output:
         J: o custo final
         theta: seu vetor de peso final
     Dica: você pode querer imprimir o custo para ter certeza de que está diminuindo.
    '''

    # # obtendo 'm', o número de linhas na matriz x
    m = len(x)
    #m = x.shape[0]
    
    for i in range(0, num_iters):
        
        # obtendo z, o produto escalar (dot product) de x e theta
        z = np.dot(x, theta)
        
        # obtendo o sigmoide de z
        h = sigmoid(z)
        
        # calculando a função custo (cost function)
        J = (-1/float(m)) * (np.dot(np.transpose(y),np.log(h)) + np.dot(np.transpose(1-y),np.log(1-h)))

        # atualizando o peso de theta
        theta = theta - (alpha/float(m)) * (np.dot(np.transpose(x),(h-y)))

    J = float(J)
    return J, theta

In [60]:
# Check the function
# Construct a synthetic test case using numpy PRNG functions
np.random.seed(1)
# X input is 10 x 3 with ones for the bias terms
tmp_X = np.append(np.ones((10, 1)), np.random.rand(10, 2) * 2000, axis=1)
# Y Labels are 10 x 1
tmp_Y = (np.random.rand(10, 1) > 0.35).astype(float)

# Apply gradient descent
tmp_J, tmp_theta = gradientDescent(tmp_X, tmp_Y, np.zeros((3, 1)), 1e-8, 700)
print(f"The cost after training is {tmp_J:.8f}.")
print(f"The resulting vector of weights is {[round(t, 8) for t in np.squeeze(tmp_theta)]}")

The cost after training is 0.67094970.
The resulting vector of weights is [4.1e-07, 0.00035658, 7.309e-05]


## Parte 2: Extraindo os recursos

* Dada uma lista de tweets, vamos extrair os recursos e armazenar em uma matriz. Vamos extrair dois recursos.
     * O primeiro recurso é o número de palavras positivas em um tweet.
     * O segundo recurso é o número de palavras negativas em um tweet.
* Em seguida, vamos treinar o classificador de regressão logística nesses recursos.
* E testar o classificador em um conjunto de validação.

### Implementando a função extract_features.
* Esta função aceita um único tweet.
* Processaremos o tweet usando a função importada `process_tweet ()` e salvaremos a lista de palavras do tweet.
* Vamos realizar um loop em cada palavra na lista de palavras processadas
     * Para cada palavra, vamos verificar o dicionário `freqs` para saber a contagem quando essa palavra tiver um rótulo '1' positivo. (Verifique a chave (word, 1.0)
     * Vamos fazer o mesmo para a contagem de quando a palavra estiver associada ao rótulo negativo '0'. (Verifique a chave (word, 0,0).)

Vamos também lidar com os casos em que a chave (word, label) não é encontrada no dicionário. Para isso utilizamos o método `.get()`, lembra?

In [25]:
def extract_features(tweet, freqs):
    '''
    Input: 
        tweet: uma lista de palavras para um tweet
        freqs: um dicionário correspondente às frequências de cada tupla (word, label)
    Output: 
        x: um vetor de features de dimensão (1,3)
    '''
    # process_tweet tokeniza, faz stemização e remove stopwords
    word_l = process_tweet(tweet)
    
    # 3 elementos (0, 0, 0) na forma de um vetor 1 x 3
    x = np.zeros((1, 3)) 
    
    #bias é 1 (primeiro elemento)
    x[0,0] = 1 
 
    # loop por cada palavra na lista de palavras
    for word in word_l:

        # para a contagem de palavras, incrementa se positivo
        x[0,1] += freqs.get((word, 1.0), 0)
        
        # para a contagem de palavras, incrementa se negativo
        x[0,2] += freqs.get((word, 0.0), 0)
        

    assert(x.shape == (1, 3))
    return x

In [26]:
# teste 1
# testando no training data
tmp1 = extract_features(train_x[0], freqs)
print(tmp1)

[[1.00e+00 3.02e+03 6.10e+01]]


In [27]:
# teste 2:
# verificando se as palavras não estão no dicionário de freqs
tmp2 = extract_features('blorb bleeeeb bloooob', freqs)
print(tmp2)

[[1. 0. 0.]]


## Parte 3: Treinando o Modelo
Para treinar o modelo:
* Vamos empilhar os recursos para todos os exemplos de treinamento em uma matriz `X`.
* E chamar `gradientDescent`, que implementamos acima.

In [28]:
# coletando os recursos 'x' e empilhando em uma matriz 'X'
X = np.zeros((len(train_x), 3))
for i in range(len(train_x)):
    X[i, :]= extract_features(train_x[i], freqs)

# as etiquetas de treinamento correspondentes a X
Y = train_y

# aplicando gradientDescent
J, theta = gradientDescent(X, Y, np.zeros((3, 1)), 1e-9, 1500)
print(f"The cost after training is {J:.8f}.")
print(f"The resulting vector of weights is {[round(t, 8) for t in np.squeeze(theta)]}")

The cost after training is 0.24216477.
The resulting vector of weights is [7e-08, 0.0005239, -0.00055517]


**Resultado esperado**: 

```
The cost after training is 0.24216529.
The resulting vector of weights is [7e-08, 0.0005239, -0.00055517]
```

# Testando a regressão logística

Vamos **testar** a função de regressão logística em alguma nova entrada que nosso modelo não viu antes e **prever** se um tweet é positivo ou negativo.

* Para cada tweet, processar e extrair os recursos.
* Aplicar os pesos aprendidos do modelo nos recursos para obter os logits.
* Aplicar o sigmóide aos logits para obter a previsão (um valor entre 0 e 1).

$$y_{pred} = sigmoid(\mathbf{x} \cdot \theta)$$

In [30]:
def predict_tweet(tweet, freqs, theta):
    '''
    Input: 
        tweet: uma string
        freqs: dicionário de frequências (word, label)
        theta: vetor de pesos (3,1) 
    Output: 
        y_pred: probabilidade de um tweet ser positivo ou negativo
    '''

    # extraindo features do tweet e colocando em "x"
    x = extract_features(tweet, freqs)
    
    # realizando a predição com x e theta utilizando .dot
    y_pred = sigmoid(np.dot(x, theta))
      
    return y_pred

In [31]:
# Vamos testar nossa função
for tweet in ['I am happy', 'I am bad', 'this movie should have been great.', 'great', 'great great', 'great great great', 'great great great great']:
    print( '%s -> %f' % (tweet, predict_tweet(tweet, freqs, theta)))

I am happy -> 0.518580
I am bad -> 0.494339
this movie should have been great. -> 0.515331
great -> 0.515464
great great -> 0.530898
great great great -> 0.546273
great great great great -> 0.561561


In [37]:
# Aqui você pode testar suas próprias frases. A função predict_tweet está funcionando!
my_tweet = 'NLP is so good'
predict_tweet(my_tweet, freqs, theta)

array([[0.51349316]])

## Verificando o desempenho usando o _test set_
Depois de treinar o modelo usando o conjunto de treinamento acima, vamos verificar como o modelo pode funcionar em dados reais e ainda não vistos, testando-o em relação ao conjunto de teste.

* Dados os dados de teste e os pesos do modelo treinado, vamos calcular a precisão do modelo de regressão logística.
* Vamos usar a função `predict_tweet ()` para fazer previsões em cada tweet no conjunto de teste.
* Se a previsão for > 0,5, a classificação do modelo `y_hat` será 1, caso contrário, será 0.
* Uma previsão é ACURADA quando `y_hat` é igual a` test_y`. Somaremos todas as instâncias quando forem iguais e divida por `m`.

Usaremos np.asarray() para converter a lista em um array numpy
Usaremos np.squeeze() para transformar o array dimensional (m,1) em (m,)

In [38]:
def test_logistic_regression(test_x, test_y, freqs, theta):
    """
    Input: 
        test_x: lista de tweets
        test_y: vetor (m, 1) com as etiquetas de x
        freqs: dicionário de frequências pos e neg
        theta: vetor de pesos de dimensão (3, 1)
    Output: 
        accuracy: (tweets classificados corretamente) / (total de tweets)
    """
    
    # lista para colocar as predições
    y_hat = []
    
    for tweet in test_x:
        # conseguindo a predição de qual etiqueta será
        y_pred = predict_tweet(tweet, freqs, theta)
        
        if y_pred > 0.5:
            # append 1.0 para a lista
            y_hat.append(1.0)
        else:
            # append 0 para a lista
            y_hat.append(0)

     # Com a implementação acima, y_hat é uma lista, mas test_y é array (m, 1), então...
     # vamos converter ambos em arrays unidimensionais para compará-los usando o operador '=='
    accuracy = sum((np.asarray(y_hat) == np.squeeze(test_y))) / len(test_y)
   
    return accuracy

In [39]:
tmp_accuracy = test_logistic_regression(test_x, test_y, freqs, theta)
print(f"Logistic regression model's accuracy = {tmp_accuracy:.4f}")

Logistic regression model's accuracy = 0.9950


# Parte 5: Análise de Erro

Nesta parte, veremos quais os tweets que nosso modelo classificou errado. 

Por que você acha que as classificações erradas aconteceram? Especificamente, que tipo de tweets o modelo classificou incorretamente?

In [41]:
print('Label Predicted Tweet')
for x,y in zip(test_x,test_y):
    y_hat = predict_tweet(x, freqs, theta)

    if np.abs(y - (y_hat > 0.5)) > 0:
        print('THE TWEET IS:', x)
        print('THE PROCESSED TWEET IS:', process_tweet(x))
        print('%d\t%0.8f\t%s' % (y, y_hat, ' '.join(process_tweet(x)).encode('ascii', 'ignore')))

Label Predicted Tweet
THE TWEET IS: @jaredNOTsubway @iluvmariah @Bravotv Then that truly is a LATERAL move! Now, we all know the Queen Bee is UPWARD BOUND : ) #MovingOnUp
THE PROCESSED TWEET IS: ['truli', 'later', 'move', 'know', 'queen', 'bee', 'upward', 'bound', 'movingonup']
1	0.49996897	b'truli later move know queen bee upward bound movingonup'
THE TWEET IS: @MarkBreech Not sure it would be good thing 4 my bottom daring 2 say 2 Miss B but Im gonna be so stubborn on mouth soaping ! #NotHavingit :p
THE PROCESSED TWEET IS: ['sure', 'would', 'good', 'thing', '4', 'bottom', 'dare', '2', 'say', '2', 'miss', 'b', 'im', 'gonna', 'stubborn', 'mouth', 'soap', 'nothavingit', ':p']
1	0.48650628	b'sure would good thing 4 bottom dare 2 say 2 miss b im gonna stubborn mouth soap nothavingit :p'
THE TWEET IS: I'm playing Brain Dots : ) #BrainDots
http://t.co/UGQzOx0huu
THE PROCESSED TWEET IS: ["i'm", 'play', 'brain', 'dot', 'braindot']
1	0.48370676	b"i'm play brain dot braindot"
THE TWEET IS: I'm p

# Parte 6: Preveja com seu próprio tweet

In [44]:
# Sinta-se à vontade para alterar o tweet abaixo
my_tweet = 'I have the impression that with the passage of time, mathematics will no longer be a nuisance, to be a great relief!'
print(process_tweet(my_tweet))
y_hat = predict_tweet(my_tweet, freqs, theta)
print(y_hat)
if y_hat > 0.5:
    print('Positive sentiment')
else: 
    print('Negative sentiment')

['impress', 'passag', 'time', 'mathemat', 'longer', 'nuisanc', 'great', 'relief']
[[0.51022075]]
Positive sentiment
